In [ ]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import json
from types import SimpleNamespace
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

%run data_preprocess.ipynb
%run helper.ipynb
%run wrapper.ipynb
%run models/UNETNilm.ipynb
%run visualize.ipynb
%run data_postprocess.ipynb

In [ ]:
dataport_appliance_data_661 = {
    "refrigerator": {
        "mean": 60.52,
        "std": 66.42,
        "window":100,
        'on_power_threshold': 50,
        'min': 0,
        'max': 683
    },
    "ev_car": {
        "mean": 204.14,
        "std": 737,
        'window':10,
        'on_power_threshold': 2000,
        'max': 3420,
        'min': 0,
    },
    "dishwasher": {
        "mean": 6.77,
        "std": 55.07,
        "window":3,
        'on_power_threshold': 10,
        'max': 960,
        'min': 0,
    },
    "clotheswasher": {
        "mean": 7.38,
        "std": 52.77,
        "window":100,
        'on_power_threshold': 20,
        'max': 1143,
        'min': 0,
    },
    "air": {
        "mean": 442.99,
        "std": 769.72,
        "window":100,
        'on_power_threshold': 20,
        'max': 2783,
        'min': 0,
    },
    "microwave": {
        "mean": 8.23,
        "std": 36.44,
        "window":100,
        'on_power_threshold': 200,
        'min': 0,
        'max': 980
    },
    "others": {
        "mean": 195,
        "std": 178,
        "window":100,
        'on_power_threshold': 0,
        'min': 0,
        'max': 1942
    }
}

# 8156
dataport_appliance_data_8156 = {
    "refrigerator": {
        "mean": 93.80,
        "std": 54.22,
        "window":100,
        'on_power_threshold': 50,
        'min': 6,
        'max': 562
    },
    "ev_car": {
        "mean": 207.76,
        "std": 750.25,
        'window':10,
        'on_power_threshold': 2000,
        'max': 3330,
        'min': 0,
    },
    "dishwasher": {
        "mean": 28.34,
        "std": 117.14,
        "window":3,
        'on_power_threshold': 10,
        'max': 936,
        'min': 0,
    },
    "clotheswasher": {
        "mean": 16.72,
        "std": 96.89,
        "window":100,
        'on_power_threshold': 20,
        'max': 1141,
        'min': 0,
    },
    "air": {
        "mean": 918.16,
        "std": 1214.59,
        "window":100,
        'on_power_threshold': 20,
        'max': 3949,
        'min': 0,
    },
    "microwave": {
        "mean": 13.89,
        "std": 73.69,
        "window":100,
        'on_power_threshold': 200,
        'min': 0,
        'max': 1678
    },
    "others": {
        "mean": 610.91,
        "std": 461.16,
        "window":100,
        'on_power_threshold': 0,
        'min': 76,
        'max': 5226
    }
}


In [ ]:
columns = ['refrigerator', 'ev_car', 'dishwasher', 'clotheswasher', 'air', 'microwave', 'others']

In [ ]:
with open("Config/model_config.json") as json_data:
    data = json.load(json_data, object_hook=lambda d: SimpleNamespace(**d))
    model_params = data.unetnilm
    model_params.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
with open("Config/appliances_list.json") as json_data:
    appliances_list = json.load(json_data)
    appliances_list['appliances'] = appliances_list['appliances']

In [ ]:
data_preprocess = QuantileDataPreprocess(model_params)

In [ ]:
visualizer = Visualizer()
data_postprocess = DataPostProcess(model_params)

In [ ]:
model_evaluation = ModelEvaluation()

In [ ]:
data_preprocess.pre_process_data_for_unetnilm(dataport_appliance_data_8156, '8156', columns)

In [ ]:
data_preprocess.pre_process_data_for_unetnilm(dataport_appliance_data_661, '661', columns)

In [ ]:
helper = PrepareDataForQuantile(model_params)

In [ ]:
train_loader, test_loader, val_loader = helper.get_data_loaders(model_params.start_col, model_params.end_col, model_params.lag_size)

In [ ]:
for x, y in train_loader:
    print(x.shape, y.shape)
    break

In [ ]:
wrapper = Wrapper(model_params)

In [ ]:
unet_model = UNETNilm(in_size=model_params.input_size, output_size=model_params.output_size, seq_len=model_params.lag_size).to(model_params.device)
# criterion =  QuantileLoss(model_params.quantiles) # Mean Squared Error loss
criterion = nn.MSELoss()
optimizer = optim.Adam(unet_model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

In [ ]:
model_evaluation.get_memory_stats(unet_model, input_shape=(30, 1), batch_size=256)

In [ ]:
ground_truth, prediction = wrapper.regression_model_train(unet_model, train_loader, val_loader, optimizer, criterion, scheduler)

In [ ]:
ground_truth_1 = ground_truth
prediction_1 = prediction

In [ ]:
ground_truth = ground_truth_1
prediction = prediction_1

In [ ]:
ground_truth, prediction = data_postprocess.reverse_scale_data_unetnilm(ground_truth.cpu().detach().numpy(), prediction.cpu().detach().numpy(), dataport_appliance_data_8156, 'standard')

In [ ]:
visualizer.visualize_plot( ground_truth, prediction, 00, 160000, 0)

In [ ]:
ground_truth_test, prediction_test = wrapper.regression_model_test(unet_model, test_loader)

In [ ]:
visualizer.visualize_plot(ground_truth_test.cpu().detach().numpy(), prediction_test.cpu().detach().numpy(), 800, 12000, 2)

In [ ]:
ground_truth_test, prediction_test = data_postprocess.reverse_scale_data_unetnilm(ground_truth_test.cpu().detach().numpy(), prediction_test.cpu().detach().numpy(), dataport_appliance_data_661, 'standard')

In [ ]:
ground_truth_test.shape, prediction_test.shape

In [ ]:
visualizer.visualize_plot(ground_truth_test, prediction_test, 200, 48000, 4)

In [ ]:
wrapper.appliance_wise_metrics(appliances_list, ground_truth_test, prediction_test)

In [ ]:
wrapper.appliance_wise_metrics(appliances_list, ground_truth_test, prediction_test)

In [ ]:
wrapper.appliance_wise_metrics(appliances_list, ground_truth_test, prediction_test)

In [ ]:
# save the predicted and ground truth values to csv files
ground_truth_df = pd.DataFrame(ground_truth_test, columns=columns)
prediction_df = pd.DataFrame(prediction_test, columns=columns)

In [ ]:
ground_truth_df.to_csv('Results/output/ground_truth_unet_withoutev_8156.csv', index=False)

In [ ]:
prediction_df.to_csv('Results/output/prediction_unet_withoutev_8156.csv', index=False)